In [1]:
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import datetime as dt
import pandas_datareader.data as web

start = dt.datetime.now()-dt.timedelta(days=15)
end = dt.datetime.now()

## Read in an old dataset from csv 

In [2]:
df_old=pd.read_csv('ML_DF_Regression.csv')
df_old.set_index('Date',inplace=True)
df_old.head(2)

,Brent,EUR_USD,Emerging_Markets,MOEXBC,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo,VOL_USD
Date,,,,,,,,,,
2019-07-31,65.02,1.10780,41.770000,18672.57,7.25,23.510000,2.03,63.6250,2.31,1.437966e+11
2019-07-30,64.05,1.11534,42.290001,18619.38,7.25,23.620001,2.04,63.5075,2.31,1.419658e+11


## Acquiring new data from Moscow Exchange and Yahoo finance  

In [3]:
Brent = web.get_data_moex('BRU9', start, end)
Brent.drop(['BOARDID', 'SECID', 'OPENPOSITIONVALUE', 'OPEN', 'LOW', 'HIGH', 'VALUE',  'OPENPOSITION','VOLUME'], 1, inplace=True)
Brent.index.name= 'Date'
Brent.sort_index(inplace=True) 
Brent.rename(columns={'CLOSE':'Brent'}, inplace=True)

EUR_USD = web.get_data_moex('EURUSD000TOM', start, end)
EUR_USD=EUR_USD.loc[EUR_USD.BOARDID=='CETS']
EUR_USD.drop(['BOARDID', 'SECID', 'SHORTNAME', 'VOLRUR', 'OPEN', 'LOW', 'HIGH', 'NUMTRADES',  'WAPRICE'], 1, inplace=True)
EUR_USD.index.name= 'Date'
EUR_USD.sort_index(inplace=True) 
EUR_USD.rename(columns={'CLOSE':'EUR_USD'}, inplace=True)

USD_RUB_New = web.get_data_moex('USD000UTSTOM', start, end)
USD_RUB_New=USD_RUB_New.loc[USD_RUB_New.BOARDID=='CETS']
USD_RUB_New.drop(['BOARDID', 'SECID', 'SHORTNAME',  'OPEN', 'WAPRICE'], 1, inplace=True)
USD_RUB_New.index.name= 'Date'
USD_RUB_New.rename(columns={'CLOSE':'USD_RUB','VOLRUR':'VOL_USD','LOW':'USD_LOW', 'HIGH':'USD_HIGH', 'NUMTRADES':'USD_NUMTRADES'}, inplace=True)
USD_RUB_New.sort_index(inplace=True) 

RGBITR_New= web.get_data_moex('RGBITR', start, end)
RGBITR_New.drop(['BOARDID', 'SECID', 'SHORTNAME', 'NAME', 'OPEN', 'LOW', 'HIGH','CLOSE','DECIMALS', 'CURRENCYID',  'DIVISOR','VALUE','DURATION','CAPITALIZATION'], 1, inplace=True)
RGBITR_New.index.name= 'Date'
RGBITR_New.index=pd.to_datetime(RGBITR_New.index)
RGBITR_New.rename(columns={'YIELD':'RGBITR_YIELD'}, inplace=True)
RGBITR_New.sort_index(inplace=True) 

USD_Repo = web.get_data_moex('MOEXREPOUSD', start, end)
USD_Repo.drop(['BOARDID', 'SECID', 'SHORTNAME', 'NAME', 'VALUE', 'OPEN', 'LOW', 'HIGH', 'DURATION', 'YIELD', 'DECIMALS', 'CAPITALIZATION', 'CURRENCYID', 'DIVISOR'], 1, inplace=True)
USD_Repo.rename(columns={'CLOSE':'USD_Repo'}, inplace=True)
USD_Repo.index.name= 'Date'
USD_Repo.index=pd.to_datetime(USD_Repo.index)
USD_Repo.sort_index(inplace=True) 

MOEXBC_New= web.get_data_moex('MOEXBC', start, end)
MOEXBC_New.drop(['BOARDID', 'SECID', 'SHORTNAME', 'NAME', 'OPEN', 'LOW', 'HIGH','DECIMALS', 'CURRENCYID',  'DIVISOR','VALUE','DURATION','CAPITALIZATION','YIELD'], 1, inplace=True)
MOEXBC_New.index.name= 'Date'
MOEXBC_New.index=pd.to_datetime(MOEXBC_New.index)
MOEXBC_New.rename(columns={'CLOSE':'MOEXBC'}, inplace=True)
MOEXBC_New.sort_index(inplace=True)

tickers = ['RSX','^IRX','EEM']
ETFs = pd.DataFrame()    

for ticker in tickers:
        ETF = web.get_data_yahoo(ticker, start, end)
        ETF.rename(columns={'Adj Close': ticker}, inplace=True)
        ETF.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)
        if ETFs.empty:
            ETFs = ETF
        else:
            ETFs = ETFs.join(ETF, how='outer')
ETFs.rename(columns={'RSX': 'Russia_ETF', '^IRX':'Treas_4M','EEM': 'Emerging_Markets'}, inplace=True)

my_data=[Brent.Brent, EUR_USD.EUR_USD, ETFs.Emerging_Markets, MOEXBC_New.MOEXBC, RGBITR_New.RGBITR_YIELD, ETFs.Russia_ETF, ETFs.Treas_4M, USD_RUB_New.USD_RUB, USD_Repo.USD_Repo, USD_RUB_New.VOL_USD]
my_data=pd.concat(my_data,axis=1)
my_data.index=pd.to_datetime(my_data.index)
my_data.tail(2)

,Brent,EUR_USD,Emerging_Markets,MOEXBC,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo,VOL_USD
Date,,,,,,,,,,
2019-07-30,64.05,1.11534,42.290001,18619.38,7.25,23.620001,2.04,63.5075,2.31,1.419658e+11
2019-07-31,65.02,1.10780,41.770000,18672.57,7.25,23.510000,2.03,63.6250,2.31,1.437966e+11


## Joining old and new datasets and saving back to csv

In [4]:
new_df = df_old.append(my_data)
new_df.index=pd.to_datetime(new_df.index)
new_df.sort_index(ascending=False,inplace=True)
new_df.drop_duplicates(subset='MOEXBC',inplace=True)
new_df.dropna(inplace=True)
new_df.to_csv('ML_DF_Regression.csv')
new_df.head(2)

,Brent,EUR_USD,Emerging_Markets,MOEXBC,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo,VOL_USD
Date,,,,,,,,,,
2019-07-31,65.02,1.10780,41.770000,18672.57,7.25,23.510000,2.03,63.6250,2.31,1.437966e+11
2019-07-30,64.05,1.11534,42.290001,18619.38,7.25,23.620001,2.04,63.5075,2.31,1.419658e+11


## Read in the first 601 rows, which are in our case the latest 601 trading days and dropping redundant features.

In [5]:
main_df=pd.read_csv('ML_DF_Regression.csv', nrows= 601)
main_df.drop(['VOL_USD','Emerging_Markets','MOEXBC'], 1,inplace=True)
main_df.reset_index(drop=True,inplace=True)
main_df.head(2)

,Date,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
0,2019-07-31,65.02,1.10780,7.25,23.510000,2.03,63.6250,2.31
1,2019-07-30,64.05,1.11534,7.25,23.620001,2.04,63.5075,2.31


In [6]:
main_df.describe()

,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
count,601.000000,601.000000,601.000000,601.000000,601.000000,601.000000,601.000000
mean,64.191398,1.153223,7.851248,20.158287,1.664068,61.622022,2.180982
std,9.465520,0.044082,0.503522,1.479707,0.587049,3.858043,0.624935
min,44.770000,1.049700,6.820000,16.566936,0.490000,55.705000,1.000000
25%,55.990000,1.125650,7.470000,19.083433,1.050000,57.840000,1.670000
50%,64.270000,1.149840,7.860000,20.064659,1.823000,62.050000,2.310000
75%,71.880000,1.179200,8.240000,21.020000,2.258000,65.340000,2.500000
max,85.900000,1.251700,9.100000,24.120001,2.408000,70.565000,4.090000


In [7]:
predict_df = main_df[(main_df.index == 0 )] # The latest trading day information available

main_df.drop('Date', 1,inplace=True)

EPOCHS =250 # how many passes through our data
BATCH_SIZE =5  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.

# opt='adam'
opt='rmsprop'

NAME = f"{BATCH_SIZE}-{opt}--{main_df.columns.tolist()}--{int(time.time())}" 

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
filepath = "regression-{epoch:02d}-{val_mean_absolute_error:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_mean_absolute_error', verbose=1, save_best_only=True, mode='min'))

tf.logging.set_verbosity(tf.logging.ERROR) # disable warnings

## Train/test split

In [8]:
test_db=main_df.loc[(main_df.index > 0) & (main_df.index <= 90)]
test_db.tail(2)

,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
89,68.35,1.1415,8.16,21.41,2.408,63.8675,2.35
90,67.63,1.1351,8.13,21.23,2.405,64.3900,2.34


In [9]:
train_df = main_df[(main_df.index > 90 )] 
train_df.head(2)

,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
91,67.52,1.13418,8.17,21.299999,2.385,64.365,2.33
92,67.07,1.13230,8.24,20.870001,2.388,64.815,2.33


## Feature-wise normalisation

In [10]:
train_targets = train_df.USD_RUB.values
train_data=train_df.drop("USD_RUB", 1).values

test_targets = test_db.USD_RUB.values
test_data=test_db.drop("USD_RUB", 1).values

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

## Building and training our network

In [11]:
%%time
model = Sequential()
model.add(Dense(128, activation='relu',input_shape=(train_data.shape[1],)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1))

model.compile(optimizer=opt, loss='mse', metrics=['mae'])

history = model.fit(
    train_data, train_targets,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(test_data, test_targets),
    callbacks=[tensorboard, checkpoint],
   )

Train on 510 samples, validate on 90 samples
Epoch 1/250
510/510 [==============================] - 6s 13ms/sample - loss: 3637.4927 - mean_absolute_error: 60.2527 - val_loss: 3816.2917 - val_mean_absolute_error: 61.7743
Epoch 2/250
510/510 [==============================] - 2s 4ms/sample - loss: 3326.6862 - mean_absolute_error: 57.6128 - val_loss: 3405.9024 - val_mean_absolute_error: 58.3586
Epoch 3/250
510/510 [==============================] - 0s 949us/sample - loss: 2883.1143 - mean_absolute_error: 53.6053 - val_loss: 2823.2705 - val_mean_absolute_error: 53.1309
Epoch 4/250
510/510 [==============================] - 1s 3ms/sample - loss: 2339.9323 - mean_absolute_error: 48.2655 - val_loss: 2183.5534 - val_mean_absolute_error: 46.7251
Epoch 5/250
510/510 [==============================] - 1s 1ms/sample - loss: 1745.0939 - mean_absolute_error: 41.6221 - val_loss: 1386.0828 - val_mean_absolute_error: 37.2254
Epoch 6/250
510/510 [==============================] - 1s 1ms/sample - loss: 

Our validation metric to monitor during training is mae (Mean Absolute Error). It is simply the absolute value of the difference between the predictions and the targets. So from the training results displayed above the best accuracy equals val_mean_absolute_error: 1.301 or 1.3 RUB which is about 2%. 

## Getting and normalizing features to predict on

In [12]:
predict_df

,Date,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
0,2019-07-31,65.02,1.1078,7.25,23.51,2.03,63.625,2.31


In [13]:
predict_data=predict_df.drop(["USD_RUB",'Date'], 1).values
prediction_target=round(float(predict_df.USD_RUB.values),2)
predict_data-= mean
predict_data /= std
predict_data

array([[ 0.13738874, -1.109691  , -1.17147689,  3.15234172,  0.81844467,
         0.22935219]])

## Load the most accurate model, ready to predict!
The idea behind this type of prediction which is more like evaluation actually since i'm not predicting future price directly is that DNN after analysed the most influential historical data and grasped fundamental dependencies is more accurate in "fair" pricing then traders. So just to be clear from this point DNN outcome is "fair" price and the actual close of a day is a deviation from that price. 

In [14]:
model = tf.keras.models.load_model("regression-247-1.301.model")
prediction =  round(float(model.predict(predict_data[:1],batch_size=1)),2)

print("Real value =",prediction_target)
print("Prediction =", prediction)  

Real value = 63.62
Prediction = 60.67


## Some simple quantitative analysis of the results.

In [15]:
test_prediction =  model.predict(test_data,batch_size=1)
deviation= abs(np.subtract(test_prediction,test_targets.reshape(-1, 1)))

mean_error = round(float(deviation.mean(axis=0)),2)
std_deviation =  round(float(deviation.std(axis=0)),2)

USD_mean = round(main_df.USD_RUB.mean(axis=0),2)
USD_std =round(main_df.USD_RUB.std(axis=0),2)
last_prediction_error= round(float(prediction-prediction_target),2)
Relative_Mispricing= round(float(last_prediction_error/std_deviation),2)

print("USD mean value =", USD_mean) # mean value for the latest 601 trading days 
print("USD standard deviation =", USD_std) # standard deviation value for the latest 601 trading days 
print("Mean error =",mean_error) #  mean error value of our prediction calculated on test dataset (90 trading days)  
print("Standard deviation of the error =", std_deviation) # standard deviation of  prediction error calculated on  test dataset (90 trading days) 
print("The latest available day prediction error =", last_prediction_error)  # just by how many RUBs we're off for the latest day
print("Relative Mispricing =", Relative_Mispricing) # by how many standard deviations we're off for the latest day

USD mean value = 61.62
USD standard deviation = 3.86
Mean error = 1.3
Standard deviation of the error = 0.98
The latest available day prediction error = -2.95
Relative Mispricing = -3.01


## Wrapping up
Current USD/RUB price is off by -3.01 standard deviations from its "fair" estimate  which is statistically, according to the Empirical Rule is extremely rare and less than 0.03% likely. Based on this model and statistical approach it's a good time to short USD futures. But i wouldn't really recommend to gamble based only on this idea.   